In [1]:
import torch
import torchsummary
import os
import sys
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn as nn

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(ROOT_DIR)

from utils import helper, input_getter, qai_hub_jobs, tfhelper
import training
import rcvit

/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 13:59:27.469758: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 13:59:27.478145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742302767.487141 2241429 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742302767.489823 2241429 cud

In [2]:
import training.lightning_model
import training.lightning_train_function

modelBase = rcvit.rcvit_t()

weights = torch.randn([64])

modelBase.dist = False
modelBase.head = nn.Linear(512, 64) # Menja classifier head da bude 64

# lightningModel = training.lightning_model.LightningModel(modelBase, class_weights=weights)
# lightningModel.load_from_checkpoint("../../models/CASVIT_T_SD_BALANCING/Ver1.0/CASVIT_T_SD_BALANCING-epoch=14-train_loss=0.0492-val_loss=0.1135.ckpt")

lightningModel = training.lightning_model.LightningModel.load_from_checkpoint("../../models/CASVIT_T_TRAIN_WITH_SD_AND_HIST_EQ/Ver1.0/CASVIT_T_TRAIN_WITH_SD_AND_HIST_EQ-epoch=32-train_loss=0.0509-val_loss=0.0870.ckpt", model = modelBase)

model = lightningModel.model
model.eval()

/home/centar15-desktop1/LPCV_2025_T1/src/training/lightning_model.py:15: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RCViT(
  (patch_embed): Sequential(
    (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (network): ModuleList(
    (0): Sequential(
      (0): AdditiveBlock(
        (local_perception): LocalIntegration(
          (network): Sequential(
            (0): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
            (3): GELU(approximate='none')
            (4): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, aff

In [3]:
model.cuda()
model.eval()

torchsummary.summary(model.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 112, 112]           1,344
       BatchNorm2d-2         [-1, 48, 112, 112]              96
              ReLU-3         [-1, 48, 112, 112]               0
            Conv2d-4           [-1, 96, 56, 56]          41,568
       BatchNorm2d-5           [-1, 96, 56, 56]             192
              ReLU-6           [-1, 96, 56, 56]               0
            Conv2d-7           [-1, 96, 56, 56]           9,312
       BatchNorm2d-8           [-1, 96, 56, 56]             192
            Conv2d-9           [-1, 96, 56, 56]             960
             GELU-10           [-1, 96, 56, 56]               0
           Conv2d-11           [-1, 96, 56, 56]           9,312
 LocalIntegration-12           [-1, 96, 56, 56]               0
      BatchNorm2d-13           [-1, 96, 56, 56]             192
           Conv2d-14          [-1, 288,

In [4]:
from dataset import DatasetReader
from dataset.utils import GLOBAL_CLASSES
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    # transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

categories = GLOBAL_CLASSES



In [5]:
import shutil
from dataset.utils import CLASSES_IMAGENET
from PIL import Image

folderpath = "/home/centar15-desktop1/Desktop/imagenet_test"
output_folder="/home/centar15-desktop1/Desktop/presek_test_imagenet"

model.eval()
model.cuda()

i = 0
p = 0

for filename in os.listdir(folderpath):
    filepath = os.path.join(folderpath, filename)
    
    try:
        image = Image.open(filepath).convert("RGB")
        image = transform(image).unsqueeze(0).cuda()  # Dodaj batch dimenziju
        
        with torch.no_grad():
            outputs = model(image)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            max_prob, predicted_class = torch.max(probabilities, dim=1)
            predicted_class = CLASSES_IMAGENET[predicted_class]
        
        # Kreiraj folder za tu klasu ako ne postoji
        
        # Premesti sliku u odgovarajući folder
        if(max_prob.item() >= 0.85):
            class_folder = os.path.join(output_folder, str(predicted_class))
            os.makedirs(class_folder, exist_ok=True)
            shutil.move(filepath, os.path.join(class_folder, filename))
            p += 1

    
    except Exception as e:
        print(f"Greška pri obradi slike {filename}: {e}")
    
    if(i%5000 == 0):
        print("Slika " + str(i))
    i += 1

print("Razvrstavanje slika završeno!")
print("Ukupno premesteno " + str(p) + " slika")


Slika 0
Slika 5000
Slika 10000
Slika 15000
Slika 20000
Slika 25000
Slika 30000
Slika 35000
Slika 40000
Slika 45000
Slika 50000
Slika 55000
Slika 60000
Slika 65000
Slika 70000
Slika 75000
Slika 80000
Slika 85000
Slika 90000
Slika 95000
Razvrstavanje slika završeno!
Ukupno premesteno 42821 slika


In [10]:
import os
import shutil

# Folderi
original_folder = "/home/centar15-desktop1/Desktop/imagenet_test"
sorted_folder = "/home/centar15-desktop1/Desktop/1kSort"

# Prolazi kroz sve podfoldere i vraća slike nazad
for class_folder in os.listdir(sorted_folder):
    class_path = os.path.join(sorted_folder, class_folder)
    
    if os.path.isdir(class_path):  # Proverava da li je to folder
        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)
            shutil.move(file_path, os.path.join(original_folder, filename))

print("Sve slike su vraćene u originalni folder!")


Sve slike su vraćene u originalni folder!


In [8]:
model1k: nn.Module = rcvit.rcvit_t()
sd = torch.load("CASVIT_t.pth")
model1k.load_state_dict(sd["model"])

/tmp/ipykernel_2241429/2204742392.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load("CASVIT_t.pth")


<All keys matched successfully>

In [11]:
folderpath = "/home/centar15-desktop1/Desktop/imagenet_test"
output_folder = "/home/centar15-desktop1/Desktop/1kSort"

model1k.eval()
model1k.cuda()

i = 0
p = 0

for filename in os.listdir(folderpath):
    filepath = os.path.join(folderpath, filename)
    
    try:
        image = Image.open(filepath).convert("RGB")
        image = transform(image).unsqueeze(0).cuda()  # Dodaj batch dimenziju
        
        with torch.no_grad():
            outputs = model1k(image)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            max_prob, predicted_class = torch.max(probabilities, dim=1)
            # predicted_class = CLASSES_IMAGENET[predicted_class]
        
        # Kreiraj folder za tu klasu ako ne postoji
        
        # Premesti sliku u odgovarajući folder
        if(max_prob.item() >= 0.5):
            class_folder = os.path.join(output_folder, str(predicted_class.item()))
            os.makedirs(class_folder, exist_ok=True)
            shutil.move(filepath, os.path.join(class_folder, filename))
            p += 1

    
    except Exception as e:
        print(f"Greška pri obradi slike {filename}: {e}")
    
    if(i%3000 == 0):
        print("Slika " + str(i))
    i += 1

print("Razvrstavanje slika završeno!")
print("Ukupno premesteno " + str(p) + " slika")


Slika 0
Slika 3000
Slika 6000
Slika 9000
Slika 12000
Slika 15000
Slika 18000
Slika 21000
Slika 24000
Slika 27000
Slika 30000
Slika 33000
Slika 36000
Slika 39000
Slika 42000
Slika 45000
Slika 48000
Slika 51000
Slika 54000
Slika 57000
Razvrstavanje slika završeno!
Ukupno premesteno 52703 slika
